Habanero is the Python library for the CrossRef API will need to be installed locally first before running notebook. More info about habanero: https://pypi.org/project/habanero/

In [ ]:
#pip install habanero 

In [ ]:
from habanero import Crossref
import csv #for handling output
from IPython.display import display
import urllib.request
import urllib.parse
from xml.etree import ElementTree as ET

In [ ]:
cr = Crossref()

Should get author info (Last name, First initial) for comparison

In [ ]:
oauthorLast = input("Enter last name):")

In [ ]:
oauthorFirst = input("Enter first name):")

In [ ]:
print(oauthorLast, oauthorFirst)

In [ ]:
inputFile = input("Input file name path (ex: ./CV/Sheldon_CV_text_short.txt)):")

In [ ]:
print(inputFile)

In [ ]:
outputFile = "output" + oauthorLast + oauthorFirst + ".txt" 

In [ ]:
print(outputFile)

In [ ]:
def cv_parse(l, r):

    cv_dict = dict()
    

    for z in x['message']['items']:
        #z['search-terms']
        
        #cit = z.get('query', None)
        cv_dict['Citation'] = l.rstrip()
        
        authors = z.get('author',None)
        if authors != None:
            authors = ""
            
            for a in z['author']:
                famname = a.get('family',None) 
                if famname == None:
                    famname = "NoName"
                givname = a.get('given',None) 
                if givname == None:
                    givname = "NoName"
                fullname = famname + ", " + givname + "; "
                #print(fullname)
                authors = authors + fullname
            if oauthor in authors:
                cv_dict['Author'] = authors
            else:
                cv_dict['Author'] = "NotAMatch"
        else:
            cv_dict['Author'] = "No Authors Listed"
        
        ppubdate = z.get('published-print',None)
        opubdate = z.get('published-online',None)
        if ppubdate != None:
            cv_dict['Year'] = str(ppubdate['date-parts'][0][0])
            if len(ppubdate['date-parts'][0]) == 1:
                cv_dict['Month-Day'] = None 
            elif len(ppubdate['date-parts'][0]) == 2:
                cv_dict['Month-Day'] = str(ppubdate['date-parts'][0][1])
            elif len(ppubdate['date-parts'][0]) == 3:
                cv_dict['Month-Day'] = str(ppubdate['date-parts'][0][1]) + "-" + str(ppubdate['date-parts'][0][2])
        elif opubdate != None:
            cv_dict['Year'] = str(opubdate['date-parts'][0][0])
            if len(opubdate['date-parts'][0]) == 1:
                cv_dict['Month-Day'] = None 
            elif len(opubdate['date-parts'][0]) == 2:
                cv_dict['Month-Day'] = str(opubdate['date-parts'][0][1])
            elif len(opubdate['date-parts'][0]) == 3:
                cv_dict['Month-Day'] = str(opubdate['date-parts'][0][1]) + "-" + str(opubdate['date-parts'][0][2])
        else:
            cv_dict['Year'] = None

        title = z.get('title', None)
        if title != None:
            cv_dict['Title'] = title[0]
        else:
            cv_dict['Title'] = None

        pubtype = z.get('type',None)
        cv_dict['Type'] = pubtype

        publisher = z.get('publisher',None)
        cv_dict['Publisher'] = publisher

        jname = z.get('container-title',None)
        if jname != None:
            cv_dict['JournalName'] = jname[0]    
        else:
            cv_dict['JournalName'] = None
            

        #cv_dict['Editor'] = editor
        
        editor = z.get('editor',None)
        if editor != None:
            editors = ""
            
            for a in z['editor']:
                famname = a.get('family',None) 
                if famname == None:
                    famname = "NoName"
                givname = a.get('given',None) 
                if givname == None:
                    givname = "NoName"
                fullname = famname + ", " + givname + "; "
                #print(fullname)
                editors = editors + fullname
                cv_dict['Editor'] = editors

        else:
            cv_dict['Editor'] = None

        volume = z.get('volume',None)
        cv_dict['JournalVolume'] = str(volume)

        #Ask about issue?
        issue = z.get('issue',None)
        cv_dict['JournalIssue'] = str(issue)

        page = z.get('page',None)
        if page != None:
            pageSplit = page.split("-")
            cv_dict['Startpage'] = str(pageSplit[0])
            if len(pageSplit) == 1:
                cv_dict['Endpage'] = None
            else:
                cv_dict['Endpage'] = str(pageSplit[1])
        else:
            cv_dict['Startpage'] = None
            cv_dict['Endpage'] = None

        doi = z.get('DOI',None)
        cv_dict['DOI'] = doi

        issn = z.get('issn-type',None)
        if issn != None:
            cv_dict['issn'] = issn[0]['value'] + ", " + issn[0]['type']
        else:
            cv_dict['issn'] = None
        
        if issn != None:
            srdict = dict()
            condlist = list()
            
            #url = 'http://www.sherpa.ac.uk/romeo/api29.php?issn=0001-6918'
            url = 'http://www.sherpa.ac.uk/romeo/api29.php?issn=' + issn[0]['value']
            #print(url)
            
            f = urllib.request.urlopen(url)
            srxml = f.read().decode('utf-8', errors='ignore')
            #print(srxml)
            root = ET.fromstring(srxml)

            for p in root.findall('.//journal'):
                srdict['journalInfo'] = "%s | %s | %s" % (p.find('jtitle').text, p.find('issn').text, p.find('zetocpub').text)

            for c in root.findall('.//publisher/conditions/condition'): # get ALL conditions make this a list then append as needed
                condlist.append(c.text)
            cv_dict['SHERPA-RoMEOc'] = condlist

            for c in root.findall('.//romeocolour'):
                srdict['colour'] = "Colour: %s" % (c.text)

            cv_dict['SHERPA-RoMEOv'] = srdict['journalInfo'] + "; " + srdict['colour']
            
            
            #cv_dict['SHERPA-RoMEO'] = url
    #writer.writerow(cv_dict)   
    return(cv_dict)

In [ ]:
with open(inputFile, "r", newline="\n") as input_file, open(outputFile, "w", newline="\n") as output_file:
    #rline = input_file.readline()
    fieldnames = ['Citation', 'Author', 'Year', 'Month-Day', 'Title', 'Type', 'Editor', 'Publisher', 'JournalName', 'JournalVolume', 'JournalIssue', 'Startpage', 'Endpage', 'DOI', 'issn', 'SHERPA-RoMEOv', 'SHERPA-RoMEOc', 'Notes']
    writer = csv.DictWriter(output_file, delimiter = '\t', fieldnames=fieldnames)
    writer.writeheader()
    #writer.writerow(['Citation', 'Author', 'Year', 'Month-Day', 'Title', 'Type', 'Editor', 'Publisher', 'JournalName', 'JournalVolume', 'JournalIssue', 'Startpage', 'Endpage', 'DOI', 'issn', 'SHERPA-RoMEO', 'Notes']) # replace with real field names
    row = 0
    for line in input_file:
        # call function to parse citations
        #print(line)
        x = cr.works(query = line, limit = 1) 
        #print(x['message'])
        cvd = cv_parse(line, x)
        #print(cvd)   
        writer.writerow(cvd)
        row = row + 1
        print(row)

In [ ]:
import pandas
df = pandas.read_csv('EDSTemplate.txt', sep='\t')
pandas.set_option('display.max_rows', 500)
pandas.set_option('display.max_colwidth', -1)
display(df)

To see individual citation results:

In [ ]:
#Optional to see individual citation results

x = cr.works(query = "Sheldon, N.D., Costa, E., Cabrera, L., and Garcés, M., 2012, Continental climatic and weathering response to the Eocene-Oliogocene transition: Journal of Geology 120, 227–236. doi: 10.1086/663984", limit = 1)
print(x['message'])

SHERPA-RoMEO API, just plug in an ISSN

In [ ]:
import urllib.request
import urllib.parse
from xml.etree import ElementTree as ET
url = 'http://www.sherpa.ac.uk/romeo/api29.php?issn=0016-7606'
f = urllib.request.urlopen(url)
srxml = f.read().decode('utf-8')
#print(srxml)
root = ET.fromstring(srxml)

for p in root.findall('.//journal'):
    print("%s | %s | %s" % (p.find('jtitle').text, p.find('issn').text, p.find('zetocpub').text))
    
for c in root.findall('.//publisher/conditions/condition'): # make this a list then append as needed
    print("Conditions: %s" % (c.text))
    
for c in root.findall('.//romeocolour'):
    print("Colour: %s" % (c.text))
